In [ ]:
import re
import math

In [ ]:
demo = False
if (demo):
    file = 'input_ex1.txt'
else:
    # Puzzle input
    file = 'input.txt'

In [ ]:
file = open(file, 'r')
input = file.readlines()
file.close()

In [ ]:
bp = {}
for line in input:
    split = re.split(' |:', line.strip())
    bp[int(split[1])] = [int(split[7]),int(split[13]),int(split[19]),int(split[22]),int(split[28]),int(split[31])]

In [ ]:
def compute_new_resources(b, type, res, rob, t):
    basic = (res[0]+rob[0]*t, res[1]+rob[1]*t, res[2]+rob[2]*t, res[3]+rob[3]*t)
    if type == "ore":
        return (basic[0]-bp[b][0], basic[1], basic[2], basic[3])
    elif type == "clay":
        return (basic[0]-bp[b][1], basic[1], basic[2], basic[3])
    elif type == "obsidian":
        return (basic[0]-bp[b][2], basic[1]-bp[b][3], basic[2], basic[3])
    elif type == "geode":
        return (basic[0]-bp[b][4], basic[1], basic[2]-bp[b][5], basic[3])
    else:
        return basic

In [ ]:
def simulate(b, res, rob, t):
    # print("Time: "+str(t))
    # print("\tRobots: "+str(rob))
    # print("\tResources: "+str(res))

    if t<=0:
        n_geodes.append(res[3])
        return

    # geode robots
    if rob[2] > 0:
        mr = [bp[b][4] - res[0], bp[b][5] - res[2]]
        tn = [1, 1]
        if max(mr) > 0:
            tn = [1+int(math.ceil(mr[0]/rob[0])),1+int(math.ceil(mr[1]/rob[2]))]
        if t>=max(tn):
            nr = compute_new_resources(b, "geode", res, rob, max(tn))
            simulate(b, nr, (rob[0], rob[1], rob[2], rob[3]+1), t-max(tn))

    # obsidian robots
    if rob[1] > 0:
        mr = [bp[b][2] - res[0], bp[b][3] - res[1]]
        tn = [1, 1]
        if max(mr) > 0:
            tn = [1+int(math.ceil(mr[0]/rob[0])),1+int(math.ceil(mr[1]/rob[1]))]
        if t>=max(tn):
            nr = compute_new_resources(b, "obsidian", res, rob, max(tn))
            simulate(b, nr, (rob[0], rob[1], rob[2]+1, rob[3]), t-max(tn))

    # clay robots
    mr = bp[b][1] - res[0]
    tn = 1
    if mr > 0:
        tn += int(math.ceil(mr/rob[0]))
    if t>=tn:
        nr = compute_new_resources(b, "clay", res, rob, tn)
        simulate(b, nr, (rob[0], rob[1]+1, rob[2], rob[3]), t-tn)
    
    # ore robots
    # reduce solution space: not more than x ore robots
    if rob[0] < 4:
        mr = bp[b][0] - res[0]
        tn = 1
        if mr > 0:
            tn += int(math.ceil(mr/rob[0]))
        if t>=tn:
            nr = compute_new_resources(b, "ore", res, rob, tn)
            simulate(b, nr, (rob[0]+1, rob[1], rob[2], rob[3]), t-tn)

    nr = compute_new_resources(b, "None", res, rob, t)
    n_geodes.append(nr[3])
    return


In [ ]:
max_geodes = {}
for b in bp.keys():
    if b == 4:
        break
    print("Blueprint "+str(b))
    n_geodes = []
    resources = (0,0,0,0)
    robots = (1,0,0,0)
    time_remaining = 32
    simulate(b, resources, robots, time_remaining)
    print("\t"+str(max(n_geodes))+" geodes obtained")
    max_geodes[b] = max(n_geodes)

In [ ]:
prod = 1
for k,v in max_geodes.items():
    prod *= v
prod
# 3168, ca. 30 min. runtime

### Demo
* BP 1: 56 with max 2 ore robots (correct)
* BP 2: 49 with max 2 ore robots (wrong)
### Puzzle
* BP 1: 6 with max 4 ore robots
* BP 2: 44 with max 3 ore robots
* BP 3: 12 with max 3 ore robots